<center><a href="https://sites.google.com/fat.uerj.br/livia/"> <img src="../images/capa2.png" alt="Header" style="width: 800px;"/> </a></center>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vnikoofard/DeepLearningTF/blob/main/notebooks/05b_presidential_doggy_door.ipynb)

# *Transfer Learning* (aprendizado por transferência)

Até agora, treinamos modelos precisos em grandes conjuntos de dados e também baixamos um modelo pré-treinado que usamos sem necessidade de treinamento. Mas e se não conseguirmos encontrar um modelo pré-treinado que faça exatamente o que você precisa e se não tivermos um conjunto de dados grande o suficiente para treinar um modelo do zero? Nesse caso, existe uma técnica muito útil que podemos usar chamada [aprendizado por transferência](https://blogs.nvidia.com/blog/2019/02/07/what-is-transfer-learning/).

Com o aprendizado por transferência, pegamos um modelo pré-treinado e o treinamos novamente em uma tarefa que tem alguma sobreposição com a tarefa de treinamento original. Uma boa analogia para isso é um artista habilidoso em um meio, como a pintura, que deseja aprender a praticar em outro meio, como o desenho a carvão. Podemos imaginar que as habilidades que aprenderam enquanto pintavam seriam muito valiosas para aprender a desenhar com carvão.

Como exemplo de aprendizado profundo, digamos que temos um modelo pré-treinado que é muito bom em reconhecer diferentes tipos de carros e queremos treinar um modelo para reconhecer tipos de motocicletas. Muitos dos aprendizados do modelo de carro provavelmente seriam muito úteis, por exemplo, a capacidade de reconhecer faróis e rodas.

O aprendizado por transferência é especialmente poderoso quando não temos um conjunto de dados grande e variado. Nesse caso, um modelo treinado do zero provavelmente memorizaria os dados de treinamento rapidamente, mas não seria capaz de generalizar bem para novos dados. Com o aprendizado por transferência, você pode aumentar suas chances de treinar um modelo preciso e robusto em um pequeno conjunto de dados.

## Objetivos

* Prepare um modelo pré-treinado para aprendizado de transferência
* Realize o aprendizado de transferência com seu próprio pequeno conjunto de dados em um modelo pré-treinado
* Ajuste ainda mais o modelo para um desempenho ainda melhor

## Uma porta personalizada para cachorros

Em nosso último exercício, usamos um modelo [ImageNet](http://www.image-net.org/) pré-treinado para deixar entrar todos os cães, mas impedir a entrada de outros animais. Neste exercício, gostaríamos de criar uma porta para cachorros que deixa entrar apenas um cachorro em particular. Neste caso, faremos uma porta automática para um cachorro chamado `Bo`, o primeiro cão dos Estados Unidos entre 2009 e 2017. Há mais fotos de Bo na pasta `data/presidential_doggy_door`.

<img src="../data/presidential_doggy_door/train/bo/bo_10.jpg">

O desafio é que o modelo pré-treinado não foi treinado para reconhecer esse cachorro específico e temos apenas 30 fotos de Bo. Se tentássemos treinar um modelo do zero usando essas 30 imagens, teríamos overfitting e generalização ruim. No entanto, se começarmos com um modelo pré-treinado que seja adepto da detecção de cães, podemos aproveitar esse aprendizado para obter uma compreensão generalizada de Bo usando nosso conjunto de dados menor. Podemos usar o aprendizado por transferência para resolver esse desafio.

## Baixando o modelo pré-treinado

Os [modelos pré-treinados do ImageNet](https://keras.io/api/applications/vgg/#vgg16-function) geralmente são boas escolhas para o aprendizado de transferência de visão computacional, pois aprenderam a classificar vários tipos diferentes de imagens. Ao fazer isso, eles aprenderam a detectar muitos tipos diferentes de [features](https://developers.google.com/machine-learning/glossary#) que podem ser valiosos no reconhecimento de imagens. Como os modelos ImageNet aprenderam a detectar animais, incluindo cães, ele é especialmente adequado para essa tarefa de aprendizado de transferência de detecção de Bo.

Vamos começar baixando o modelo pré-treinado. Novamente, isso está disponível diretamente na biblioteca Keras. Como estamos baixando, haverá uma diferença importante. A última camada de um modelo ImageNet é uma [camada Dense](https://developers.google.com/machine-learning/glossary#dense-layer) de 1.000 unidades, representando as 1.000 classes possíveis no conjunto de dados. No nosso caso, queremos que faça uma classificação diferente: isso é Bo ou não? Como queremos que a classificação seja diferente, vamos remover a última camada do modelo. Podemos fazer isso definindo o sinalizador `include_top=False` ao baixar o modelo. Depois de remover esta camada superior, podemos adicionar novas camadas que resultarão no tipo de classificação que desejamos:

In [ ]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=False)

In [ ]:
base_model.summary()

## Congelando o Modelo Base
Antes de adicionarmos nossas novas camadas ao [modelo pré-treinado](https://developers.google.com/machine-learning/glossary#pre-trained-model), devemos dar um passo importante: congelar o modelo pré-treinado camadas treinadas. Isso significa que quando treinamos, não atualizaremos as camadas base do modelo pré-treinado. Em vez disso, atualizaremos apenas as novas camadas que adicionamos no final para nossa nova classificação. Congelamos as camadas iniciais porque queremos reter o aprendizado obtido com o treinamento no conjunto de dados ImageNet. Se eles fossem descongelados neste estágio, provavelmente destruiríamos essas informações valiosas. Haverá uma opção para descongelar e treinar essas camadas posteriormente, em um processo chamado de ajuste fino.

Congelar as camadas de base é tão simples quanto definir treinável no modelo para `False`.

In [ ]:
base_model.trainable = False

## Adicionando novas camadas

Agora podemos adicionar as novas camadas treináveis ao modelo pré-treinado. Eles pegarão os recursos das camadas pré-treinadas e os transformarão em previsões no novo conjunto de dados. Vamos adicionar duas camadas ao modelo. A primeira será uma camada de agrupamento como vimos em nossa [rede neural convolucional](https://developers.google.com/machine-learning/glossary#convolutional_layer) anterior. (Se você quiser uma compreensão mais completa do papel das camadas de pooling em CNNs, leia [esta postagem de blog detalhada](https://machinelearningmastery.com/pooling-layers-for-convolutional-neural-networks/#:~: text=A%20pooling%20layer%20is%20a,Convolutional%20Layer)). Em seguida, precisamos adicionar nossa camada final, que classificará Bo ou não Bo. Esta será uma camada densamente conectada (*densely connected*) com uma saída.

In [ ]:
inputs = keras.Input(shape=(224, 224, 3))
# Separadamente da configuração treinável no modelo, definimos o treinamento como Falso
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
# Um classificador denso com uma única unidade (classificação binária)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

Vamos dar uma olhada no modelo, agora que combinamos o modelo pré-treinado com as novas camadas.

In [ ]:
model.summary()

Keras nos dá um bom resumo aqui, pois mostra o modelo pré-treinado vgg16 como uma unidade, em vez de mostrar todas as camadas internas. Também é importante notar que temos muitos parâmetros não treináveis, pois congelamos o modelo pré-treinado.

## Compilando o Modelo

Como em nossos exercícios anteriores, precisamos compilar o modelo com opções de perda e métricas. Temos que fazer algumas escolhas diferentes aqui. Nos casos anteriores, tínhamos muitas categorias em nosso problema de classificação. Como resultado, escolhemos a entropia cruzada categórica para o cálculo de nossa perda. Neste caso, temos apenas um problema de classificação binária (Bo ou não Bo), e por isso usaremos [binary crossentropy](https://www.tensorflow.org/api_docs/python/tf/keras/losses/BinaryCrossentropy). Mais detalhes sobre as diferenças entre os dois podem ser encontrados [aqui](https://gombru.github.io/2018/05/23/cross_entropy_loss/). Também usaremos precisão binária em vez da precisão tradicional.

Ao definir `from_logits=True`, informamos a [função de perda](https://gombru.github.io/2018/05/23/cross_entropy_loss/) que os valores de saída não são normalizados (por exemplo, com softmax).

In [ ]:
# Importante usar entropia binária e precisão binária, pois agora temos um problema de classificação binária
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True), metrics=[keras.metrics.BinaryAccuracy()])

## Aumentando os dados

Agora que estamos lidando com um conjunto de dados muito pequeno, é especialmente importante aumentarmos nossos dados. Como antes, faremos pequenas modificações nas imagens existentes, o que permitirá ao modelo ver uma variedade maior de imagens para aprender. Isso o ajudará a aprender a reconhecer novas imagens de Bo, em vez de apenas memorizar as imagens nas quais treina.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Criar um gerador de dados
datagen_train = ImageDataGenerator(
    samplewise_center=True,  # define a média de cada amostra para 0
    rotation_range=10,  # gira aleatoriamente as imagens no intervalo (graus, 0 a 180)
    zoom_range=0.1,  # Amplie a imagem aleatoriamente
    width_shift_range=0.1,  # muda aleatoriamente as imagens horizontalmente (fração da largura total)
    height_shift_range=0.1,  # muda aleatoriamente as imagens verticalmente (fração da altura total)
    horizontal_flip=True, # virar imagens aleatoriamente
    vertical_flip=False,
)  # não esperamos que Bo esteja de cabeça para baixo, então não vamos virar verticalmente

# Não há necessidade de aumentar os dados de validação
datagen_valid = ImageDataGenerator(samplewise_center=True)

## Carregando os dados

Vimos conjuntos de dados em alguns formatos diferentes até agora. No exercício MNIST, conseguimos baixar o conjunto de dados diretamente da biblioteca Keras. Para o conjunto de dados de língua de sinais, os dados estavam em arquivos CSV. Para este exercício, vamos carregar imagens diretamente de pastas usando a função [`flow_from_directory`](https://keras.io/api/preprocessing/image/) do Keras. Configuramos nossos diretórios para ajudar esse processo a ocorrer sem problemas, pois nossos rótulos são inferidos a partir dos nomes das pastas. No diretório `data/presidential_doggy_door`, temos diretórios de treinamento e validação, cada um com pastas para imagens de Bo e não de Bo. Nos diretórios not_bo, temos fotos de outros cães e gatos, para ensinar nosso modelo a afastar outros animais de estimação. Sinta-se à vontade para explorar as imagens para ter uma noção do nosso conjunto de dados.

Observe que [flow_from_directory](https://keras.io/api/preprocessing/image/) também nos permitirá dimensionar nossas imagens para corresponder ao modelo: 244x244 pixels com 3 canais.

In [ ]:
# essa celula é necesaria para manter a compatibilidade em rodar o codigo no colab
import sys
import shutil

def download_github_folder(repository_url, folder_path):
    # Clone the repository
    repo = Repo.clone_from(repository_url, './temp_repo')

    # Get the path to the folder within the cloned repository
    folder_repo_path = repo.working_dir + '/' + folder_path

    # Perform the download using a file copy
    shutil.copytree(folder_repo_path, './downloaded_folder')

    # Remove the temporary repository folder
    shutil.rmtree('./temp_repo')



if "google.colab" in sys.modules:
    %pip install gitpython
    from git import Repo
    download_github_folder('https://github.com/vnikoofard/DeepLearningTF.git', '')
    shutil.copytree('downloaded_folder/data/presidential_doggy_door/', './', dirs_exist_ok=True)
    file_path = "presidential_doggy_door/"
else:
    file_path = "../data/presidential_doggy_door/"

In [ ]:
# load and iterate training dataset
train_it = datagen_train.flow_from_directory(
    file_path + "train/",
    target_size=(224, 224),
    color_mode="rgb",
    class_mode="binary",
    batch_size=8,
)

# load and iterate validation dataset
valid_it = datagen_valid.flow_from_directory(
    file_path + "valid/",
    target_size=(224, 224),
    color_mode="rgb",
    class_mode="binary",
    batch_size=8,
)

## Treinando o Modelo

Hora de treinar nosso modelo e ver como ele se sai. Lembre-se que ao usar um gerador de dados, temos que definir explicitamente o número de `steps_per_epoch`:

In [ ]:
model.fit(train_it, steps_per_epoch=12, validation_data=valid_it, validation_steps=4, epochs=20)

## Discussão dos Resultados

Tanto a precisão de treinamento quanto a de validação devem ser bastante altas. Este é um resultado bastante impressionante! Conseguimos treinar em um pequeno conjunto de dados, mas devido ao conhecimento transferido do modelo ImageNet, ele conseguiu obter alta precisão e generalizar bem. Isso significa que ele tem um bom senso de Bo e de animais de estimação que não são Bo.

Se você viu alguma flutuação na precisão da validação, tudo bem também. Temos uma técnica para melhorar nosso modelo na próxima seção.

## Ajustando o Modelo (*Fine-Tuning*)

Agora que as novas camadas do modelo estão treinadas, temos a opção de aplicar um truque final para melhorar o modelo, chamado [fine-tuning](https://developers.google.com/machine-learning/glossary#f) . Para fazer isso, descongelamos todo o modelo e o treinamos novamente com uma [taxa de aprendizado] muito pequena (https://developers.google.com/machine-learning/glossary#learning-rate). Isso fará com que as camadas básicas pré-treinadas deem passos muito pequenos e se ajustem ligeiramente, melhorando o modelo em uma pequena quantidade.

Observe que é importante executar esta etapa somente após o modelo com camadas congeladas ter sido totalmente treinado. As camadas de agrupamento e classificação não treinadas que adicionamos ao modelo anteriormente foram inicializadas aleatoriamente. Isso significa que eles precisavam ser atualizados bastante para classificar corretamente as imagens. Por meio do processo de [backpropagation](https://developers.google.com/machine-learning/glossary#backpropagation), grandes atualizações iniciais nas últimas camadas teriam causado atualizações potencialmente grandes também nas camadas pré-treinadas. Essas atualizações teriam destruído esses importantes recursos pré-treinados. No entanto, agora que essas camadas finais foram treinadas e convergiram, quaisquer atualizações no modelo como um todo serão muito menores (especialmente com uma taxa de aprendizado muito pequena) e não destruirão os recursos das camadas anteriores.

Vamos tentar descongelar as camadas pré-treinadas e ajustar o modelo:

In [ ]:
# Descongelar o modelo base
base_model.trainable = True

# É importante recompilar seu modelo depois de fazer qualquer alteração
# ao atributo `treinável` de qualquer camada interna, para que suas alterações
# sejam levados em consideração
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .00001),  # Very low learning rate
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])

In [ ]:
model.fit(train_it, steps_per_epoch=12, validation_data=valid_it, validation_steps=4, epochs=10)

## Examinando as previsões

Agora que temos um modelo bem treinado, é hora de criar nossa porta doggy para Bo! Podemos começar observando as previsões que vêm do modelo. Vamos pré-processar a imagem da mesma forma que fizemos para a nossa última porta de cachorro.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing import image as image_utils
from tensorflow.keras.applications.imagenet_utils import preprocess_input

def show_image(image_path):
    image = mpimg.imread(image_path)
    plt.imshow(image)

def make_predictions(image_path):
    show_image(image_path)
    image = image_utils.load_img(image_path, target_size=(224, 224))
    image = image_utils.img_to_array(image)
    image = image.reshape(1,224,224,3)
    image = preprocess_input(image)
    preds = model.predict(image)
    return preds

Experimente isso em algumas imagens para ver as previsões:

In [ ]:
make_predictions(file_path + 'valid/bo/bo_20.jpg')

In [ ]:
make_predictions(file_path + 'valid/not_bo/121.jpg')

Parece que uma previsão de número negativo significa que é Bo e uma previsão de número positivo significa que é outra coisa. Podemos usar essa informação para que nossa porta cachorrinha só deixe Bo entrar!

## Exercício: Bo's Doggy Door

Preencha o código a seguir para implementar a porta cachorrinha de Bo:

In [ ]:
def presidential_doggy_door(image_path):
    preds = make_predictions(image_path)
    if FIXME:
        print("It's Bo! Let him in!")
    else:
        print("That's not Bo! Stay out!")

## Solução

Clique no '...' abaixo para ver a solução.

In [ ]:
# SOLUTION
def presidential_doggy_door(image_path):
    preds = make_predictions(image_path)
    if preds[0] < 0:
        print("It's Bo! Let him in!")
    else:
        print("That's not Bo! Stay out!")

Vamos experimentar!

In [ ]:
presidential_doggy_door(file_path + 'valid/not_bo/131.jpg')

In [ ]:
presidential_doggy_door(file_path + 'valid/bo/bo_29.jpg')

## Resumo

Ótimo trabalho! Com o aprendizado por transferência, você construiu um modelo altamente preciso usando um conjunto de dados muito pequeno. Essa pode ser uma técnica extremamente poderosa e ser a diferença entre um projeto bem-sucedido e um que não consegue decolar. Esperamos que essas técnicas possam ajudá-lo em situações semelhantes no futuro!

Há uma riqueza de recursos úteis para transferência de aprendizado no [NVIDIA TAO Toolkit](https://developer.nvidia.com/tlt-getting-started).

### Resetar o ambiente
Antes de prosseguir, execute a célula a seguir para resetar o ambiente. Isso é necessário para passar para o próximo notebook.

In [ ]:
import os
os._exit(00)

## Next

Até agora, o foco deste workshop tem sido principalmente na classificação de imagens. Na próxima seção, para fornecer uma introdução mais completa ao aprendizado profundo, vamos mudar de assunto e abordar o trabalho com dados sequenciais, o que requer uma abordagem diferente.